# SQLAlchemy
[SQLAlchemy](https://www.sqlalchemy.org/) è una libreria di Python che consente di interfacciarsi con diversi database per recuperare dati da utilizzare all'interno di un'applicazione.

Per interrogare un database abbiamo due modalità:
 * RAW Query
 * Object Relational Mapping
 
Supponiamo di voler definire questo semplice schema:

&nbsp;&nbsp;&nbsp;studente (<u>sid</u>, cf, nome, cognome)\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **AK**: cf

&nbsp;&nbsp;&nbsp;corso (<u>cid</u>, nome)

&nbsp;&nbsp;&nbsp;esame (<u>data, sid, cid</u>, voto)\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **FK**: sid **REFERENCES** studente\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **FK**: cid **REFERENCES** corso

Iniziamo importando la libreria di SQLAlchemy

In [3]:
import sqlalchemy

# Modalità RAW
La modalità raw consente di eseguire istruzioni SQL pure, scrivendole direttamente.

In SQLALchemy si utilizza la funzione [text](https://docs.sqlalchemy.org/en/14/core/sqlelement.html#sqlalchemy.sql.expression.text) per eseguire istruzioni SQL.

## Creare il database e definire una connessione

Utilizziamo PostgreSQL come DBMS.

1. Aprire pgAdmin
2. Creare un nuovo database denominato `test_raw`
3. Si può usare l'utente di default `postgresql` oppure creare uno nuovo

Una volta terminate le operazioni, possiamo creare la stringa di connessione al database

In [6]:
dbuser = "user"            # Nome utente
dbpass = "passwd"          # Password
dbname = "test_raw"        # Nome del database
dbhost = "127.0.0.1:5432"  # Host del database

# Definizione della connessione
engine = sqlalchemy.create_engine("postgresql://"+dbuser+":"+dbpass+"@"+dbhost+"/"+dbname)

## Creazione delle tabelle

In [7]:
# Apre la connessione
conn = engine.connect()

# Esegue le istruzioni SQL: crea le tabelle
conn.execute(sqlalchemy.text("""
    CREATE TABLE studente (
        sid INT PRIMARY KEY,
        cf CHAR(16) NOT NULL UNIQUE,
        nome VARCHAR(255) NOT NULL,
        cognome VARCHAR(255) NOT NULL
    );
    
    CREATE TABLE corso (
        cid INT PRIMARY KEY,
        nome VARCHAR(255) NOT NULL
    );
    
    CREATE TABLE esame (
        data DATE NOT NULL,
        sid INT NOT NULL REFERENCES studente(sid),
        cid INT NOT NULL REFERENCES corso(cid),
        voto INT NOT NULL,
        PRIMARY KEY(data, sid, cid)
    );
"""))

# Chiude la connessione
conn.close()

&Egrave; possibile verificare da pgAdmin se la creazione delle tabelle è andata a buon fine.

## Inserimento dei dati

Inserimento dei dati: inseriamo alcuni record

In [9]:
# Apre la connessione
conn = engine.connect()

# Esegue le istruzioni SQL: crea le tabelle
conn.execute(sqlalchemy.text("""
    INSERT INTO studente(sid, cf, nome, cognome) VALUES (1, '12345', 'Mario', 'Rossi');
    INSERT INTO corso(cid, nome) VALUES (1, 'Basi di dati');
    INSERT INTO esame(data, sid, cid, voto) VALUES ('06/06/2021', 1, 1, 25);
"""))

# Chiude la connessione
conn.close()

## Lettura dei dati
Possiamo leggere i dati dal database:

In [15]:
# Apre la connessione
conn = engine.connect()

# E' possibile parametrizzare una query in questo modo
results = conn.execute(sqlalchemy.text("SELECT * FROM studente WHERE nome = :nome"), nome='Mario')

# Chiude la connessione
conn.close()

# Per ogni record restituito mostra il contenuto
for row in results:
    # Ogni record è un dictionary a cui si può accedere per leggere il valore del campo
    print(f"sid={row['sid']}, cf={row['cf'].strip()}, nome={row['nome']}, cognome={row['cognome']}")

sid=1, cf=12345, nome=Mario, cognome=Rossi


# Modalità Object Relational Mapping (ORM)
L'ORM è una tecnica di programmazione che astrae il livello fisico del DBMS e rappresenta le tabelle sottoforma di classi, ogni record inserito in una tabella viene rappresentato come un oggetto.

In questo modo è il software a gestire la creazione del database e non ci si deve preoccupare della portabilità se si cambia il DBMS.

In [20]:
from sqlalchemy import Column, ForeignKey, Integer, String, Date, PrimaryKeyConstraint, select
from sqlalchemy.orm import declarative_base, relationship, Session

## Creare il database e definire una connessione

Utilizziamo PostgreSQL come DBMS.

1. Aprire pgAdmin
2. Creare un nuovo database denominato `test_orm`
3. Si può usare l'utente di default `postgresql` oppure creare uno nuovo

Una volta terminate le operazioni, possiamo creare la stringa di connessione al database

In [30]:
dbuser = "user"            # Nome utente
dbpass = "passwd"          # Password
dbname = "test_orm"        # Nome del database
dbhost = "127.0.0.1:5432"  # Host del database

# Definizione della connessione
engine = sqlalchemy.create_engine("postgresql://"+dbuser+":"+dbpass+"@"+dbhost+"/"+dbname)

## Creazione delle tabelle
Innanzi tutto bisogna definire la classe base che SQLAlchemy usa per definire gli oggetti che rappresentano le tabelle

In [31]:
Base = declarative_base()

Ogni tabella deve essere definita come una classe.
SQLAlchemy offre vari [tipi di dato](https://docs.sqlalchemy.org/en/14/core/type_basics.html) e la possibilità di esprimere tutti i [vincoli presenti in SQL](https://docs.sqlalchemy.org/en/14/core/constraints.html).

In [32]:
# Tabella Studente
class Studente(Base):
    __tablename__ = "studente"
    sid = Column(Integer, primary_key=True)
    # Definisce il vincolo di AK
    cf = Column(String(16), nullable=False, unique=True)
    nome = Column(String, nullable=False)
    cognome = Column(String, nullable=False)
    
    def __repr__(self):
        """
        Metodo utilizzato quando si vuole visualizzare un oggetto di tipo studente
        """
        return f"sid={self.sid}, cf={self.cf}, nome={self.nome}, cognome={self.cognome}"

class Corso(Base):
    __tablename__ = "corso"
    cid = Column(Integer, primary_key=True)
    nome = Column(String, nullable=False)
    
    def __repr__(self):
        return f"cid={self.cid}, nome={self.nome}"

class Esame(Base):
    __tablename__ = "esame"
    data = Column(Date, nullable=False)
    # Definisce anche il vincolo di FK
    sid = Column(Integer, ForeignKey("studente.sid"), nullable=False)
    cid = Column(Integer, ForeignKey("corso.cid"), nullable=False)
    voto = Column(Integer, nullable=False)
    
    # Definise la primary key su più attributi
    PrimaryKeyConstraint(data, sid, cid)
    
    def __repr__(self):
        return f"data={self.data}, sid={self.sid}, cid={self.cid}, voto={self.voto}"
    

Una volta create le tabelle, queste devono essere scritte sul database eseguendo la seguente istruzione

In [33]:
Base.metadata.create_all(engine)

&Egrave; possibile verificare con pgAdmin la corretta creazione delle tabelle e dei vincoli.

## Inserimento dei dati

Innanzi tutto bisogna stabilire una nuova sessione con il database

In [34]:
session = Session(engine)

Poi si possono creare i record, ogni record è un'istanza della classe in cui deve essere inserito

In [35]:
s1 = Studente(
    sid = 1,
    cf = "123456",
    nome = "Mario",
    cognome = "Rossi"
)
c1 = Corso(
    cid = 1,
    nome = "Basi di dati"
)
e1 = Esame(
    data = "06/06/2021",
    sid = 1,
    cid = 1,
    voto = 25
)

Per inserire un record bisogna aggiungerlo alla sessione corrente e poi chiamare l'istruzione `commit` per scrivere i dati sul database.

In [36]:
session.add_all([s1, c1])
session.commit()

session.add(e1)
session.commit()

## Interrogare il database
Le query devono essere composte con l'istruzione `select`, SQLAlchemy offre vari [metodi](https://docs.sqlalchemy.org/en/14/core/selectable.html) per creare query avanzate

In [37]:
query = select(Studente).where(Studente.cognome == "Rossi")

Il risultato di una query è un elenco di oggetti della classe che definisce la tabella

In [38]:
for studente in session.execute(query):
    print(studente[0])

sid=1, cf=123456, nome=Mario, cognome=Rossi


Esempio di query con JOIN che restituisce più oggetti, ogni record è un oggetto della classe contenuta nella select

In [39]:
query = select(Studente, Esame, Corso)\
       .join_from(Studente, Esame, Studente.sid == Esame.sid)\
       .join_from(Esame, Corso, Esame.cid == Corso.cid)

In [41]:
for row in session.execute(query):
    print(f"{row[0].nome} {row[0].cognome}")
    print(f"{row[2].nome}")
    print(f"{row[1].data} - {row[1].voto}")

Mario Rossi
Basi di dati
2021-06-06 - 25
